In [1]:
"""
references:
https://www.tensorflow.org/get_started/tflearn
https://github.com/tensorflow/tensorflow/blob/r1.2/tensorflow/examples/tutorials/monitors/iris_monitors.py

TensorFlow¡¯s high-level machine learning API (tf.contrib.learn) makes it easy to configure, train, and evaluate 
a variety of machine learning models. 

In this tutorial, you¡¯ll use tf.contrib.learn to construct a neural network classifier and train it 
on the Iris data set to predict flower species based on sepal/petal geometry. 
You'll write code to perform the following five steps:

Load CSVs containing Iris training/test data into a TensorFlow Dataset
Construct a neural network classifier
Fit the model using the training data
Evaluate the accuracy of the model
Classify new samples

"""
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
from __future__ import print_function

import os.path
import sys
import time

import urllib

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'

# Environments

In [2]:
"""
Please note, the environments for the codes.
"""

import platform

print(platform.linux_distribution())

print(platform.python_version())

print(tf.__version__)

print(np.__version__)

('Ubuntu', '14.04', 'trusty')
2.7.6
1.0.1
1.13.1


# Download Data

In [2]:
IRIS_TRAINING = "./iris_data/iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "./iris_data/iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"


# If the training and test sets aren't stored locally, download them.
if not os.path.exists(IRIS_TRAINING):
    raw = urllib.urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING, "w") as f:
        f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = urllib.urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST, "w") as f:
        f.write(raw)

# Load Data

In [3]:
# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=IRIS_TRAINING,
      target_dtype=np.int,
      features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=IRIS_TEST,
      target_dtype=np.int,
      features_dtype=np.float32)

# Specify that all features have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

In [4]:
print(type(feature_columns))
print(feature_columns)

<type 'list'>
[_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None)]


In [5]:
import pandas as pd


df = pd.read_csv(IRIS_TRAINING)
print(df)

     120    4  setosa  versicolor  virginica
0    6.4  2.8     5.6         2.2          2
1    5.0  2.3     3.3         1.0          1
2    4.9  2.5     4.5         1.7          2
3    4.9  3.1     1.5         0.1          0
4    5.7  3.8     1.7         0.3          0
5    4.4  3.2     1.3         0.2          0
6    5.4  3.4     1.5         0.4          0
7    6.9  3.1     5.1         2.3          2
8    6.7  3.1     4.4         1.4          1
9    5.1  3.7     1.5         0.4          0
10   5.2  2.7     3.9         1.4          1
11   6.9  3.1     4.9         1.5          1
12   5.8  4.0     1.2         0.2          0
13   5.4  3.9     1.7         0.4          0
14   7.7  3.8     6.7         2.2          2
15   6.3  3.3     4.7         1.6          1
16   6.8  3.2     5.9         2.3          2
17   7.6  3.0     6.6         2.1          2
18   6.4  3.2     5.3         2.3          2
19   5.7  4.4     1.5         0.4          0
20   6.7  3.3     5.7         2.1          2
21   6.4  

# Validation Metrics

In [9]:
validation_metrics = {
      "accuracy":
          tf.contrib.learn.MetricSpec(
              metric_fn=tf.contrib.metrics.streaming_accuracy,
              prediction_key="classes"),
      "precision":
          tf.contrib.learn.MetricSpec(
              metric_fn=tf.contrib.metrics.streaming_precision,
              prediction_key="classes"),
      "recall":
          tf.contrib.learn.MetricSpec(
              metric_fn=tf.contrib.metrics.streaming_recall,
              prediction_key="classes")
  }
validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
      test_set.data,
      test_set.target,
      every_n_steps=50,
      metrics=validation_metrics,
      early_stopping_metric="loss",
      early_stopping_metric_minimize=True,
      early_stopping_rounds=200)

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


# Model

In [14]:
# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                              hidden_units=[10, 20, 10],
                                              n_classes=3,
                                              model_dir="./iris_model",
                                              config = tf.contrib.learn.RunConfig(save_checkpoints_secs = 1))

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 1, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4049e077d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


# Training

In [23]:
tf.reset_default_graph()
# Fit model.
classifier.fit(x=training_set.data,
                   y=training_set.target,
                   steps=2000,
                   monitors=[validation_monitor])

# Evaluate accuracy.
accuracy_score = classifier.evaluate(x=test_set.data,
                                         y=test_set.target)["accuracy"]
print('Accuracy: {0:f}'.format(accuracy_score))

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Sav

In [24]:
# Classify two new flower samples.
new_samples = np.array(
        [[6.4, 3.2, 4.5, 1.5], [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)

y = list(classifier.predict(new_samples, as_iterable=True))
print('Predictions: {}'.format(str(y)))

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Predictions: [1, 1]
